In [3]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# path = '/content/drive/MyDrive/BTP_Major/small' #small
path = '/content/drive/MyDrive/BTP_Minor/large' #large
# path = './small'

In [4]:
symbols = pd.read_csv(path + "/sp500.csv")
symbols = symbols.drop(['Unnamed: 0','Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded'], axis=1)
symbols

,Symbol
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN
...,...
498,YUM
499,ZBRA
500,ZBH
501,ZION


In [5]:
not_included = ['CE','KVUE','VLTO','CEG','GEHC','PEAK','WBD']
symbols = symbols[~symbols['Symbol'].isin(not_included)]
symbols
vertices = symbols['Symbol']

In [ ]:
import torch
tensor_list = []
y_tensor_list = []

# not_included = ['CE','KVUE','VLTO','CEG','GEHC','PEAK','WBD']

for feature in symbols['Symbol']:
    # if feature in not_included:
    #    continue

    # stock_price = pd.read_csv(path + f"/raw_price_data/{feature}_prices.csv").set_index('Unnamed: 0') #small
    stock_price = pd.read_csv(path + f"/Large_Price_Data/{feature}_prices.csv").set_index('Date').drop(['Unnamed: 0'], axis = 1) #large

    #Normalisation
    stock_price_normalized = (stock_price - stock_price.min()) / (stock_price.max() - stock_price.min())
    stock_price = stock_price_normalized
    #seperating training data
    stock_price = stock_price[stock_price.index < '2023-05-01']
    y = stock_price['R-0']
    stock_price = stock_price.drop(['R-0'], axis = 1)
    # print(stock_price.columns)
    pv = torch.tensor(stock_price.values, dtype=torch.float)
    pv = torch.unsqueeze(pv, dim=-1)

    y_tensor = torch.tensor(y.values, dtype=torch.float)
    y_tensor = torch.unsqueeze(y_tensor, dim=-1)
    y_tensor = torch.unsqueeze(y_tensor, dim=1)

    tensor_list.append(pv)
    y_tensor_list.append(y_tensor)

    # print(feature, sep = ' ')
    # break

X_train = torch.cat(tensor_list, dim=-1)
y_train = torch.cat(y_tensor_list, dim=-1)

print(X_train.shape)
print (y_train.shape)

torch.Size([293, 49, 496])
torch.Size([293, 1, 496])


In [ ]:
# from transformers import PatchTSTForClassification
from transformers import PatchTSTConfig, PatchTSTForPrediction

In [ ]:
channels = X_train.shape[-1]

config = PatchTSTConfig(
    num_input_channels=channels,
    dropout=0.3,
    head_dropout=0.3,
    ignore_mismatched_sizes=True,
    context_length=49,
    prediction_length = 1,
    # return_dict = True,
    output_hidden_states = True

)

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "mps" if torch.backends.mps.is_built() else device
device

'cuda:0'

In [ ]:
channels = X_train.shape[-1]
# channels = 1

#Finetuning Model
model = PatchTSTForPrediction.from_pretrained(
    "namctin/patchtst_etth1_pretrain",
    num_input_channels=channels,
    dropout=0.3,
    head_dropout=0.3,
    ignore_mismatched_sizes=True,
    context_length=49,
    prediction_length = 1,
    # return_dict = True,
    output_hidden_states = True,
    # share_embedding = False
).to(device)

# #Zero Shot Prediction
# model = PatchTSTForPrediction(
#     config=config
# ).to(device)


# model = PatchTSTForClassification.from_pretrained(
#     "namctin/patchtst_etth1_pretrain",
#     num_input_channels=channels,
#     num_targets=3,
#     dropout=0.2,
#     head_dropout=0.2,
#     ignore_mismatched_sizes=True,
#     context_length=50)
model.config

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.43M [00:00<?, ?B/s]

Some weights of PatchTSTForPrediction were not initialized from the model checkpoint at namctin/patchtst_etth1_pretrain and are newly initialized: ['head.projection.bias', 'head.projection.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PatchTSTForPrediction were not initialized from the model checkpoint at namctin/patchtst_etth1_pretrain and are newly initialized because the shapes did not match:
- model.encoder.positional_encoder.position_enc: found shape torch.Size([43, 128]) in the checkpoint and torch.Size([5, 128]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PatchTSTConfig {
  "_name_or_path": "namctin/patchtst_etth1_pretrain",
  "activation_function": "gelu",
  "architectures": [
    "PatchTSTForPretraining"
  ],
  "attention_dropout": 0.0,
  "bias": true,
  "channel_attention": false,
  "channel_consistent_masking": false,
  "context_length": 49,
  "d_model": 128,
  "distribution_output": "student_t",
  "do_mask_input": null,
  "dropout": 0.3,
  "ff_dropout": 0.0,
  "ffn_dim": 512,
  "head_dropout": 0.3,
  "init_std": 0.02,
  "loss": "mse",
  "mask_input": true,
  "mask_type": "forecast",
  "mask_value": 0,
  "model_type": "patchtst",
  "norm_eps": 1e-05,
  "norm_type": "batchnorm",
  "num_attention_heads": 16,
  "num_forecast_mask_patches": 5,
  "num_hidden_layers": 3,
  "num_input_channels": 496,
  "num_parallel_samples": 100,
  "num_targets": 1,
  "output_hidden_states": true,
  "output_range": null,
  "patch_length": 12,
  "patch_stride": 12,
  "path_dropout": 0.0,
  "pooling_type": null,
  "positional_dropout": 0.0,
  "positional_en

In [ ]:
X_train.shape[0]

293

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import one_hot
import pandas as pd

# Define a custom dataset
class DataFrameDataset(Dataset):
    def __init__(self, past, future):
        self.past_values = past.to(device)
        self.future = future.to(device)

    def __len__(self):
        return self.past_values.shape[0]

    def __getitem__(self, idx):
        pv = self.past_values[idx]
        fv = self.future[idx]

        return {
            'past_values': pv,
            'future_values': fv
        }


# Create an instance of the DataFrameDataset
dataset = DataFrameDataset(X_train, y_train)

# Create a DataLoader for the dataset
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for batch in dataloader:
    print(batch['past_values'].shape, batch['future_values'].shape)
    # print(batch['labels'].shape)
    outputs = model(past_values=batch['past_values'], future_values=batch['future_values'])
    # print(outputs.prediction_logits.shape)
    # print(outputs.last_hidden_state.shape)
    print(outputs.loss)
    break


torch.Size([32, 49, 496]) torch.Size([32, 1, 496])
tensor(0.0195, device='cuda:0', grad_fn=<MseLossBackward0>)


In [ ]:
loss_epoch = list()

In [ ]:
total_loss = 0
for batch in dataloader:
    outputs = model(past_values=batch['past_values'], future_values=batch['future_values'])
    loss = outputs.loss
    total_loss += loss.item()

avg_loss = total_loss / len(dataloader)
loss_epoch.append((0, avg_loss))
best_avg_loss = avg_loss
print(best_avg_loss)

0.01898108059540391


In [ ]:
loss_epoch

[(0, 0.01898108059540391)]

In [ ]:
import torch
from torch.utils.data import DataLoader

# Define your dataset and DataLoader
# For demonstration, let's assume you have your own dataset and DataLoader setup

# Initialize the model and tokenizer
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.000000001)
loss_fn = torch.nn.MSELoss()

model.zero_grad()

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        optimizer.zero_grad()

        outputs = model(past_values=batch['past_values'], future_values=batch['future_values'])
        loss = outputs.loss
        # logits = outputs.logits
        # logits = outputs.prediction_logits

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss}")

    if avg_loss < best_avg_loss:
      best_avg_loss = avg_loss
      print(f"Model checkpoint saved at Epoch {epoch+1}")
      output_dir = './saved_model'
      model.save_pretrained(output_dir)

    loss_epoch.append((epoch+1, avg_loss))

# Optionally, you can evaluate the model on a validation set after each epoch
model.eval()
# Remember to switch to evaluation mode during validation

Epoch 1/20, Loss: 0.017006622161716223
Model checkpoint saved at Epoch 1
Epoch 2/20, Loss: 0.016738484892994166
Model checkpoint saved at Epoch 2
Epoch 3/20, Loss: 0.017270963080227374
Epoch 4/20, Loss: 0.01780608855187893
Epoch 5/20, Loss: 0.017757355328649284
Epoch 6/20, Loss: 0.01792788626626134
Epoch 7/20, Loss: 0.017061121948063373
Epoch 8/20, Loss: 0.017277107667177917
Epoch 9/20, Loss: 0.01731066908687353
Epoch 10/20, Loss: 0.017307840567082167
Epoch 11/20, Loss: 0.01665790369734168
Model checkpoint saved at Epoch 11
Epoch 12/20, Loss: 0.017132403794676065
Epoch 13/20, Loss: 0.01771914679557085
Epoch 14/20, Loss: 0.01684120548889041
Epoch 15/20, Loss: 0.01780757522210479
Epoch 16/20, Loss: 0.0170550019480288
Epoch 17/20, Loss: 0.0185426683165133
Epoch 18/20, Loss: 0.0173355002887547
Epoch 19/20, Loss: 0.01812331108376384
Epoch 20/20, Loss: 0.017756162211298943


PatchTSTForPrediction(
  (model): PatchTSTModel(
    (scaler): PatchTSTScaler(
      (scaler): PatchTSTStdScaler()
    )
    (patchifier): PatchTSTPatchify()
    (masking): Identity()
    (encoder): PatchTSTEncoder(
      (embedder): PatchTSTEmbedding(
        (input_embedding): Linear(in_features=12, out_features=128, bias=True)
      )
      (positional_encoder): PatchTSTPositionalEncoding(
        (positional_dropout): Identity()
      )
      (layers): ModuleList(
        (0-2): 3 x PatchTSTEncoderLayer(
          (self_attn): PatchTSTAttention(
            (k_proj): Linear(in_features=128, out_features=128, bias=True)
            (v_proj): Linear(in_features=128, out_features=128, bias=True)
            (q_proj): Linear(in_features=128, out_features=128, bias=True)
            (out_proj): Linear(in_features=128, out_features=128, bias=True)
          )
          (dropout_path1): Identity()
          (norm_sublayer1): PatchTSTBatchNorm(
            (batchnorm): BatchNorm1d(128, eps

In [ ]:
# outputs = model(past_values=X_train)
# outputs.hidden_states[-1].shape # Shape (days, channels, embedding size [5, 128])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_CUDA_mm)

In [ ]:
# output_dir = './saved_model'
# model.save_pretrained(output_dir)

In [ ]:
import csv
with open("train_log.csv", 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows([("epoch", "loss")])
    csv_writer.writerows(loss_epoch)

In [ ]:
!zip -r /content/saved_model.zip /content/saved_model

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/generation_config.json (deflated 15%)
  adding: content/saved_model/config.json (deflated 57%)
  adding: content/saved_model/model.safetensors (deflated 8%)


In [ ]:
# outputs = model(
#     past_values=torch.unsqueeze(torch.tensor(X_train.values, dtype=torch.float), dim=-1),
#     target_values=torch.tensor(y_train.values)
# )

In [ ]:
# loss_fn = torch.nn.CrossEntropyLoss()
# loss = loss_fn(outputs.prediction_logits, torch.tensor(y_train.values))
# loss

In [ ]:
# torch.tensor(X_train.values.shape)

In [ ]:
# print (target_values)

In [ ]:
# hs = outputs.hidden_states
# hs_list = [x for x in hs]
# print(len(hs_list), hs_list[2].shape)

In [ ]:
# loss = outputs.loss
# loss

In [ ]:
# loss.backward()

In [ ]:
# optimizer.step()

In [ ]:
# from datasets import load_dataset
# datasetDict = load_dataset("csv", data_files="MSFT_prices.csv")
# datasetDict = datasetDict.remove_columns(['Unnamed: 0'])
# dataset = datasetDict['train']
# dataset

In [ ]:
# dataset.add_column(name="past_values", column=dataset['R-49'])


In [ ]:
# from transformers import Trainer
# finetune_forecast_trainer = Trainer(
#     model=model,
#     args=finetune_forecast_args,
#     train_dataset=dataset,
#     # past_values=past_values,
#     # eval_dataset=valid_dataset,
#     callbacks=[early_stopping_callback],
# )

In [ ]:
# finetune_forecast_trainer.train()

In [ ]:
# print(outputs.last_hidden_state.shape)

#### INFERENCE

In [4]:
from transformers import PatchTSTModel

# model = PatchTSTModel.from_pretrained("/content/drive/MyDrive/BTP_Major/large/patchTST_finetune/saved_model")
model = PatchTSTModel.from_pretrained("./saved_model")

model

NameError: name '_C' is not defined

In [6]:
import torch
tensor_list = []

dates = None

for feature in symbols['Symbol']:

    # stock_price = pd.read_csv(path + f"/raw_price_data/{feature}_prices.csv").set_index('Unnamed: 0') #small
    stock_price = pd.read_csv(path + f"/Large_Price_Data/{feature}_prices.csv").set_index('Date').drop(['Unnamed: 0'], axis = 1) #large

    if dates is None:
        dates = stock_price.index
    elif (dates != stock_price.index).any():
        print(f"Index Mismatch for {feature}")
        break

    #Normalisation
    stock_price_normalized = (stock_price - stock_price.min()) / (stock_price.max() - stock_price.min())
    stock_price = stock_price_normalized
    #seperating training data
    stock_price = stock_price.drop(['R-0'], axis = 1)
    pv = torch.tensor(stock_price.values, dtype=torch.float)
    pv = torch.unsqueeze(pv, dim=-1)

    tensor_list.append(pv)

    break

X = torch.cat(tensor_list, dim=-1)

print(X.shape)

torch.Size([421, 49, 1])


In [ ]:
torch.save(X, 'X.pt')

In [8]:
pd.Series(dates,index=dates).to_csv('dates.csv', header=False, index=False)

In [ ]:
outputs = None

In [ ]:
outputs = model(X)
hs = outputs.last_hidden_state.detach()
hs.shape

In [ ]:
# Create a dataframe with x rows and y columns
# x = outputs.last_hidden_state.detach().numpy().shape[0]
# y = outputs.last_hidden_state.detach().numpy().shape[1]
df = pd.DataFrame(index=dates, columns=vertices)

# Initialize each entry with a tensor of size z
for i in range(len(dates)):
    for j in range(len(vertices)):
        df.iloc[i, j] = outputs.last_hidden_state.detach().numpy()[i, j]

df = df.map(lambda x: x.reshape(-1))
df

Symbol,ALK,LNT,GOOG,AAPL,DUK,EBAY,MSFT,MRNA,NXPI,LUV
Unnamed: 0,,,,,,,,,,
2022-03-01,"[-0.09667873, 0.5986906, -0.6772077, -0.331996...","[-0.112278685, -0.41217428, -0.31780213, 0.049...","[-0.13511913, 0.2757166, 0.12760574, -0.140742...","[-0.32096523, 0.3744733, 0.18259612, -0.325829...","[-0.28141692, -0.120090954, -0.29268682, -0.15...","[-0.38906342, 0.3202899, 0.37953803, 0.1109729...","[-0.21744601, 0.60647243, 0.2284654, -0.460996...","[-0.13210258, -0.0024422505, 0.13223888, 0.064...","[-0.45623845, -0.15623237, 0.21846944, -0.0283...","[-0.074371874, 0.310641, -0.446356, -0.1659434..."
2022-03-02,"[0.09919448, 1.016114, 0.39318702, -0.79442745...","[-0.15626828, 0.5474256, -0.46968296, 0.203388...","[-0.18734947, 1.0901734, -0.4113501, -0.375976...","[-0.20114803, 1.2909791, 0.017762087, -0.77809...","[-0.086130545, 0.7108166, -0.73323953, 0.15512...","[-0.12698448, 0.29317814, 0.029091917, -0.2592...","[0.014150187, 0.9096479, -0.25234097, -0.25101...","[-0.20239934, 0.4281972, -0.17718203, -0.34841...","[-0.11174809, 0.6643838, 0.042060375, -0.42211...","[0.038345978, 1.0352502, 0.2981011, -0.7158230..."
2022-03-03,"[0.15288042, 0.27427867, -0.022812843, 0.17453...","[-0.4250868, 0.9382821, 0.8180717, -0.65586436...","[0.14567578, 0.8899504, -0.14693803, -0.390216...","[0.15497711, 0.7967688, -0.12543994, -0.306249...","[-0.3509845, 1.0089635, 0.45453846, -0.6132697...","[0.2868945, 0.3204611, -0.46373564, 0.22777776...","[0.18323784, 0.6211307, -0.08914887, -0.229364...","[0.23863399, 0.30709034, -0.034259014, -0.3993...","[-0.08990226, 0.8468494, 0.15489534, -0.420192...","[0.022774152, 0.37216887, 0.0094046, 0.0249934..."
2022-03-04,"[0.6164063, -0.23151153, -0.10564314, 1.117966...","[0.038023893, -0.18709259, 0.24462904, 0.32850...","[0.15974002, 0.004899837, -0.3088786, 0.675225...","[0.57641584, 0.025270857, -0.2482847, 1.218084...","[-0.016813818, -0.44030362, 0.1336155, 0.39448...","[0.05584936, 0.11586468, -0.44001076, 0.693510...","[0.3048501, -0.3375629, -0.5412189, 1.0577796,...","[0.18811306, 0.3420562, -0.61877584, 0.4541822...","[0.5176942, -0.011627793, -0.19821763, 0.97058...","[0.6276673, -0.15922411, -0.13056912, 0.999137..."
2022-03-07,"[-0.20285602, -0.10872213, -0.30419353, 0.9045...","[0.17615455, -0.007802412, -0.2736806, 0.57833...","[-0.16632341, -0.023024224, -0.4286044, -0.014...","[-0.29130575, -0.7232282, -0.22195913, 0.63679...","[0.13374248, -0.33559155, -0.2255226, 0.529582...","[-0.4900679, -0.41793013, 0.4812187, 0.0613562...","[-0.16584054, -0.32582545, -0.24542499, 0.1901...","[-0.018903628, -0.15108271, -0.036239788, 0.19...","[-0.0026752818, -0.26349306, 0.22470605, 1.026...","[-0.19000292, -0.17497832, -0.13960542, 1.0063..."
...,...,...,...,...,...,...,...,...,...,...
2023-10-25,"[-0.28195453, 0.4480663, -0.6217949, -0.149121...","[-0.31115434, 0.5034908, -0.26372072, -0.12108...","[0.014472336, 0.20195292, 0.27267137, -0.19378...","[0.32081333, -0.47686145, 0.2383213, 0.0397835...","[-0.394776, 0.61689436, 0.081199236, -0.013151...","[-0.34412238, -0.39512894, 0.37076962, 0.73743...","[0.1837622, -0.07207769, 0.22492433, 0.2900956...","[0.07563747, 0.14094123, 0.24452245, 0.2262846...","[0.14646956, 0.5588133, -0.15681274, -0.265429...","[-0.16850898, -0.10201835, -0.3646658, 0.00616..."
2023-10-26,"[0.104703665, 0.64226556, 0.021257877, -0.1560...","[0.22680874, -0.07364491, -0.6453216, 0.041738...","[0.26472583, 0.29137117, -0.25347674, 0.463618...","[0.07974626, -0.22562197, 0.11836242, 0.221589...","[0.15499234, -0.1629326, -0.4252311, 0.1475214...","[-0.15041876, 0.045346916, -0.67344886, 0.0900...","[-0.5285766, 0.15222923, -0.35535145, 0.001792...","[0.2853227, 0.19052644, -0.35720184, 0.5753128...","[0.240384, 0.3051331, -0.7927903, 0.36394966, ...","[0.123267055, -0.19739988, 0.39313722, 0.52032..."
2023-10-27,"[-0.35154828, 0.25966156, 0.80480945, 0.001529...","[0.048218906, -0.3967014, 0.43688166, 0.178834...","[0.018108353, 0.6798761, 0.4692859, 0.23714676...

In [ ]:
df['ALK'][0].shape

(640,)

In [ ]:
df_train = df[df.index < '2023-05-01']
df_val = df[(df.index >= '2023-05-01') & (df.index <= '2023-06-30')]
df_test = df[df.index >= '2023-07-01']

In [ ]:
df_train

Symbol,ALK,LNT,GOOG,AAPL,DUK,EBAY,MSFT,MRNA,NXPI,LUV
Unnamed: 0,,,,,,,,,,
2022-03-01,"[-0.09667873, 0.5986906, -0.6772077, -0.331996...","[-0.112278685, -0.41217428, -0.31780213, 0.049...","[-0.13511913, 0.2757166, 0.12760574, -0.140742...","[-0.32096523, 0.3744733, 0.18259612, -0.325829...","[-0.28141692, -0.120090954, -0.29268682, -0.15...","[-0.38906342, 0.3202899, 0.37953803, 0.1109729...","[-0.21744601, 0.60647243, 0.2284654, -0.460996...","[-0.13210258, -0.0024422505, 0.13223888, 0.064...","[-0.45623845, -0.15623237, 0.21846944, -0.0283...","[-0.074371874, 0.310641, -0.446356, -0.1659434..."
2022-03-02,"[0.09919448, 1.016114, 0.39318702, -0.79442745...","[-0.15626828, 0.5474256, -0.46968296, 0.203388...","[-0.18734947, 1.0901734, -0.4113501, -0.375976...","[-0.20114803, 1.2909791, 0.017762087, -0.77809...","[-0.086130545, 0.7108166, -0.73323953, 0.15512...","[-0.12698448, 0.29317814, 0.029091917, -0.2592...","[0.014150187, 0.9096479, -0.25234097, -0.25101...","[-0.20239934, 0.4281972, -0.17718203, -0.34841...","[-0.11174809, 0.6643838, 0.042060375, -0.42211...","[0.038345978, 1.0352502, 0.2981011, -0.7158230..."
2022-03-03,"[0.15288042, 0.27427867, -0.022812843, 0.17453...","[-0.4250868, 0.9382821, 0.8180717, -0.65586436...","[0.14567578, 0.8899504, -0.14693803, -0.390216...","[0.15497711, 0.7967688, -0.12543994, -0.306249...","[-0.3509845, 1.0089635, 0.45453846, -0.6132697...","[0.2868945, 0.3204611, -0.46373564, 0.22777776...","[0.18323784, 0.6211307, -0.08914887, -0.229364...","[0.23863399, 0.30709034, -0.034259014, -0.3993...","[-0.08990226, 0.8468494, 0.15489534, -0.420192...","[0.022774152, 0.37216887, 0.0094046, 0.0249934..."
2022-03-04,"[0.6164063, -0.23151153, -0.10564314, 1.117966...","[0.038023893, -0.18709259, 0.24462904, 0.32850...","[0.15974002, 0.004899837, -0.3088786, 0.675225...","[0.57641584, 0.025270857, -0.2482847, 1.218084...","[-0.016813818, -0.44030362, 0.1336155, 0.39448...","[0.05584936, 0.11586468, -0.44001076, 0.693510...","[0.3048501, -0.3375629, -0.5412189, 1.0577796,...","[0.18811306, 0.3420562, -0.61877584, 0.4541822...","[0.5176942, -0.011627793, -0.19821763, 0.97058...","[0.6276673, -0.15922411, -0.13056912, 0.999137..."
2022-03-07,"[-0.20285602, -0.10872213, -0.30419353, 0.9045...","[0.17615455, -0.007802412, -0.2736806, 0.57833...","[-0.16632341, -0.023024224, -0.4286044, -0.014...","[-0.29130575, -0.7232282, -0.22195913, 0.63679...","[0.13374248, -0.33559155, -0.2255226, 0.529582...","[-0.4900679, -0.41793013, 0.4812187, 0.0613562...","[-0.16584054, -0.32582545, -0.24542499, 0.1901...","[-0.018903628, -0.15108271, -0.036239788, 0.19...","[-0.0026752818, -0.26349306, 0.22470605, 1.026...","[-0.19000292, -0.17497832, -0.13960542, 1.0063..."
...,...,...,...,...,...,...,...,...,...,...
2023-04-24,"[0.28025466, 0.60496354, -0.33811146, -0.31974...","[0.48448512, -0.3346718, 0.18127204, 0.3023218...","[0.36889946, 0.4722314, -0.978626, -0.58151174...","[0.68668735, 0.016696826, -0.28907332, -0.0786...","[0.48158664, -0.013480824, 0.12858957, -0.0274...","[0.02266495, 0.06954736, -0.6316679, -0.142354...","[0.67961687, -0.27926496, -0.26348865, -0.0770...","[0.27032217, 0.49498582, -0.83188385, -0.13320...","[0.52277464, -0.09155381, 0.2298778, 0.0095788...","[0.43963194, 0.41505486, -0.24581946, -0.22692..."
2023-04-25,"[0.084470674, 0.09218138, -0.102503754, 0.1853...","[-0.59224045, -1.0053791, 0.7792717, 0.9483574...","[0.35546374, -0.17419209, -0.33107656, -0.1037...","[0.33792612, -0.537189, 0.26474065, 0.8594919,...","[-0.47423473, -0.9042307, 0.8504015, 1.0465086...","[-0.124490045, -0.5530305, -0.18553466, 0.3221...","[0.46541986, -0.5809348, -0.0433082, 1.1649132...","[-0.02925393, 0.1087165, -0.28469858, 0.136524...","[0.21117872, -0.40666357, 0.25128046, 0.846444...","[0.052651867, -0.1596405, -0.19714378, 0.66773..."
2023-04-26,"[0.00042283535, -0.113418534, -0.02876858, 0.0...","[-0.108975045, -0.38540632, -0.599748, -0.0094...","[0.17660995, -0.8228342, 0.10774782, 0.0422321...

In [ ]:
df_val

Symbol,ALK,LNT,GOOG,AAPL,DUK,EBAY,MSFT,MRNA,NXPI,LUV
Unnamed: 0,,,,,,,,,,
2023-05-01,"[-0.073416844, -0.19968437, -0.24451837, -0.02...","[-0.3080237, -0.14302576, 0.523106, 0.30762002...","[-0.26241636, -0.41092324, -0.081503384, 0.285...","[-0.35182825, 0.4114321, 0.17052177, -0.577112...","[-0.31060895, -0.123406515, 0.7224273, 0.16679...","[-0.09906936, -0.80220973, -0.05786169, 0.6391...","[-0.3179121, 0.26809943, -0.04152888, -0.68284...","[-0.33159366, -0.06723778, -0.09519971, 0.2664...","[-0.2367021, 0.14481917, 0.24228862, -0.321307...","[-0.2768402, -0.15720202, 0.2033679, 0.0986627..."
2023-05-02,"[0.06000389, -0.71002996, 0.26762164, 0.309519...","[0.28502232, -0.10906509, -0.42286614, 0.54238...","[0.118136965, -0.5078715, 0.53804386, 0.360522...","[0.5585759, -0.06287023, 0.26662368, -0.239418...","[0.3267402, -0.31013107, -0.18022905, 0.503924...","[0.021854289, -0.46263885, 0.17489433, 0.44780...","[0.19166699, 0.3951149, 0.017702535, -0.721908...","[0.13564944, -0.779276, 0.10059496, 0.38633215...","[0.058236986, 0.57428336, -0.31676847, -0.4883...","[-0.011266872, -0.3724939, 0.067816086, 0.2143..."
2023-05-03,"[-0.33271962, -0.12531327, -0.011276923, 0.172...","[-0.30051613, 0.8065357, -0.7576662, 0.1217177...","[-0.20907551, 0.31482756, 0.34457964, 0.057428...","[0.2886676, 0.44405445, -0.19540375, 0.3142210...","[-0.09253128, 0.54865324, -0.7775646, 0.439429...","[-0.52413356, 0.5482963, -0.058141395, -0.1898...","[0.5427495, 0.27507138, 0.19893968, 0.06106620...","[-0.23417571, 0.024965271, -0.17866091, 0.1185...","[0.09273657, 0.20678705, -0.36810598, -0.05401...","[-0.3668902, 0.06752664, -0.42461163, 0.369322..."
2023-05-04,"[-0.017287835, -0.16351536, 0.055955455, -0.04...","[-0.43522474, 0.4679899, 0.1124808, -0.6765039...","[-0.20372906, 0.71425986, -0.07050326, -0.1476...","[-0.14789207, 0.17558414, -0.2542979, -0.00964...","[-0.4575137, 0.39917585, 0.10683292, -0.494857...","[-0.050150648, 0.45381626, 0.3175074, -0.67479...","[-0.086223334, -0.06927388, 0.041344345, 0.603...","[-0.10126199, 0.5840576, -0.3035316, -0.585202...","[0.08777173, -0.46608746, -0.19954218, 0.06481...","[-0.43975037, 0.2913, -0.140919, -0.32218364, ..."
2023-05-05,"[0.22254154, 0.476226, -0.012820926, 0.1383607...","[0.25767836, -0.42116648, 0.096420914, 0.44011...","[0.12485775, 0.72644246, -0.25694916, 0.178972...","[0.23629403, -0.551927, -0.06398437, 0.4254737...","[0.09138672, -0.33531803, 0.13300422, 0.520962...","[0.2966932, -0.081237756, 0.42480987, 0.160219...","[-0.12215921, -0.25522304, -0.43401152, 0.3663...","[0.29075125, 0.13784957, -0.17243727, 0.013163...","[0.26041088, -0.6452831, -0.027584724, -0.0119...","[0.29398218, -0.013246294, -0.1897639, -0.0475..."
2023-05-08,"[-0.27218354, 0.19006816, 0.07678542, 0.216348...","[-0.043703943, -0.18167463, -0.117877215, -0.1...","[-0.34406698, 0.6208596, -0.43647987, 0.161493...","[-0.4152651, -0.80452275, 0.2321873, 0.4959592...","[-0.15193005, -0.2641287, 0.13584375, 0.048811...","[-0.34851384, 0.38428813, -0.34111652, 0.35537...","[-0.15562677, -0.37895387, -0.29052657, 0.1788...","[-0.053279534, -0.22100903, -0.039448917, 0.43...","[0.0041707605, -0.58249795, -0.085421726, 0.07...","[0.043596327, 0.09633078, 0.44744515, 0.107938..."
2023-05-09,"[-0.28307796, -0.03937619, -0.32201993, 0.0426...","[0.09509806, 0.28845525, -1.2745509, -0.260301...","[-0.44507933, 0.8524939, -0.75466883, -0.90071...","[-0.36945906, 0.19016925, -0.66450346, -0.5554...","[-0.042903513, 0.55407715, -1.1927116, -0.4286...","[-0.19790317, 0.44350198, -0.8342755, -0.58533...","[-0.46808985, 0.19293448, -0.113971666, -0.336...","[-0.056936964, 0.20087688, -0.5809245, -0.2224...","[-0.033557296, -0.36410362, -0.3256262, 0.3017...","[-0.1742406, 0.074524805, -0.67051995, 0.15056..."
2023-05-10,"[0.01041618, 0.083179794, -0.16084526, -0.0990...","[0.5528607, -0.30142468, 0.2933915, 0.02190274...","[0.2567913, 0.31400484, -0.97066736, -0.391439...","[0.4053393, 0.21613666, -0.15272309, -0.3

In [ ]:
df_test

Symbol,ALK,LNT,GOOG,AAPL,DUK,EBAY,MSFT,MRNA,NXPI,LUV
Unnamed: 0,,,,,,,,,,
2023-07-03,"[-0.5987413, -0.8618307, 0.09885128, 0.2357822...","[0.17228585, -0.028254107, -0.07236056, 0.3395...","[-0.62165844, 0.41293544, -0.56649506, 0.21144...","[-0.28636643, 0.32687902, -0.25240073, 0.01480...","[0.09269175, 0.07551016, 0.02711625, 0.1793721...","[-0.2147333, 0.008863196, -0.0029946445, 0.143...","[-0.6094454, 0.8184395, -0.06791349, -0.303370...","[0.19894323, 0.114237465, 0.100319415, -0.3630...","[-0.16074368, -0.86276877, 0.38758802, 0.60843...","[-0.111616105, -0.4253191, -0.25496337, 0.1911..."
2023-07-05,"[-0.030582003, -0.24514614, -0.053537697, -0.1...","[0.10918216, -0.8089125, 0.3692476, 1.1624324,...","[-0.23060092, 0.3672994, -0.20260282, -0.62091...","[-0.15095074, 0.21851766, 0.13596708, -0.58780...","[0.24346694, -0.5708014, 0.16211636, 0.7672855...","[-0.23948927, -0.13910894, 0.41654545, -0.1917...","[0.26382855, -0.089328766, -0.23674102, -0.537...","[0.18674015, -0.31389263, 0.25645208, 0.404137...","[-0.1971086, -0.7459167, -0.13535786, 0.306171...","[-0.34178707, -0.5886213, 0.20237154, -0.00887..."
2023-07-06,"[0.04214777, 0.16613992, 0.41596854, -0.299950...","[-0.48938265, -0.4966544, -0.41222498, 0.43801...","[0.20072502, -0.07906506, -0.08755977, -0.5202...","[0.39102063, -0.08011552, 0.14603952, -0.02528...","[-0.22023506, -0.40661615, -0.53067887, 0.7293...","[0.36267567, -0.3414585, -0.009926528, 0.44226...","[0.37519237, -0.23574382, 0.012031481, 0.14110...","[0.13164616, -0.7462769, 0.48702657, 0.9060744...","[0.005607873, -0.17241052, -0.20528565, -0.068...","[-0.1443529, -0.1821263, -0.0334934, -0.263102..."
2023-07-07,"[-0.026441619, 0.4161806, 0.09865039, -0.04193...","[-0.19694588, 0.1944885, -0.36100888, -0.59679...","[0.36620268, 0.15317945, -0.5855031, 0.0525204...","[0.42563003, -0.17536774, -0.15721257, 0.71644...","[-0.1196278, 0.101402506, -0.2536725, -0.46578...","[0.27858505, 0.18692768, -0.017756268, 0.63185...","[-0.022505224, -0.11569071, -0.70468843, 0.651...","[-0.24546579, 0.0043613273, -0.61866915, 0.384...","[-0.24597174, 0.46792096, 0.100842774, -0.4049...","[0.22444466, 0.30149552, 0.32674018, -0.570530..."
2023-07-10,"[0.19670567, 0.110557616, -0.123253, 0.6219809...","[0.117150836, -0.03409713, -0.14663391, 0.4082...","[0.3046903, 0.18179873, -0.39668673, 0.1658229...","[-0.19079316, -0.1531315, -0.35847464, 0.49112...","[0.003514707, -0.20315044, -0.30444327, 0.4164...","[-0.55087876, 0.19294378, -0.036134977, 0.4651...","[0.2275742, -0.21943271, -0.1240578, 0.3896998...","[-0.3033187, 0.54657376, -0.32457814, -0.10999...","[0.04809569, -0.065968044, 0.29028803, -0.1779...","[0.28173947, -0.10201623, -0.040931795, 0.2424..."
...,...,...,...,...,...,...,...,...,...,...
2023-10-25,"[-0.28195453, 0.4480663, -0.6217949, -0.149121...","[-0.31115434, 0.5034908, -0.26372072, -0.12108...","[0.014472336, 0.20195292, 0.27267137, -0.19378...","[0.32081333, -0.47686145, 0.2383213, 0.0397835...","[-0.394776, 0.61689436, 0.081199236, -0.013151...","[-0.34412238, -0.39512894, 0.37076962, 0.73743...","[0.1837622, -0.07207769, 0.22492433, 0.2900956...","[0.07563747, 0.14094123, 0.24452245, 0.2262846...","[0.14646956, 0.5588133, -0.15681274, -0.265429...","[-0.16850898, -0.10201835, -0.3646658, 0.00616..."
2023-10-26,"[0.104703665, 0.64226556, 0.021257877, -0.1560...","[0.22680874, -0.07364491, -0.6453216, 0.041738...","[0.26472583, 0.29137117, -0.25347674, 0.463618...","[0.07974626, -0.22562197, 0.11836242, 0.221589...","[0.15499234, -0.1629326, -0.4252311, 0.1475214...","[-0.15041876, 0.045346916, -0.67344886, 0.0900...","[-0.5285766, 0.15222923, -0.35535145, 0.001792...","[0.2853227, 0.19052644, -0.35720184, 0.5753128...","[0.240384, 0.3051331, -0.7927903, 0.36394966, ...","[0.123267055, -0.19739988, 0.39313722, 0.52032..."
2023-10-27,"[-0.35154828, 0.25966156, 0.80480945, 0.001529...","[0.048218906, -0.3967014, 0.43688166, 0.178834...","[0.018108353, 0.6798761, 0.4692859, 0.23714676...